# Inspecting FCC Political File Data (with [`agate`](https://agate.readthedocs.io/en/1.6.1/))

In [1]:
import agate
import re

In [2]:
fcc_files = agate.Table.from_csv('fcc_opif.facilityfile.csv')

## General filters

We need to all of these Political Files into to separate piles:

1. Agreements
2. Contracts

Ideally, we would be able to derive rules for each of these filters that would apply across all of the facilities.

So far, our best guess is to rely on the folder path and file name.

In [3]:
def is_agreement(row):
    contains_agreement = (
        'agreement' in row['folder'].lower() or 
        'agreement' in row['file_name'].lower()
    )
    contains_nab = (
        'nab' in row['folder'].lower() or
        'nab' in row['file_name'].lower() or
        'n a b' in row['file_name'].lower()
    )
    return contains_agreement or contains_nab

In [4]:
def is_contract(row):
    order_in_folder_name = 'order' in row['folder'].lower()
    invoice_in_folder_name = 'invoice' in row['folder'].lower()
    # file_name contains a number matching the pattern #####--#    
    number_w_dashes_pattern = re.match(r'(.+)?\d{5}--\d+', row['file_name'])
    # file_name contains a number matching the pattern ##### (#)    
    number_w_paren_pattern = re.match(r'(.+)?\d{5}--\(\d+\)', row['file_name'])
    return (
        order_in_folder_name
        or invoice_in_folder_name
        or number_w_dashes_pattern
        or number_w_paren_pattern
    ) and not is_agreement(row)

## KOMU's Political Files

Let's identify the different kinds of files KOMU has been uploading into the Political Files parent folder.

In [5]:
komu_files = fcc_files.where(lambda row: row['folder__entity'] == 'KOMU-TV')

In [6]:
len(komu_files)

748

### Agreement Forms 

First, there are agreement forms for political advertisements signed by both the facility and represenstatives of the political campaigns. The agreement could be with candidate campaign, such as [this one](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2016/State/PARSON%20FOR%20LT%20GOVERNOR%20OF%20MISSOURI/NAB_FORM.pdf) on behalf of Governor Mike Parson, or non-candidate issue campaign, such as [this one](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2014/Non-Candidate%20Issue%20Ads/PROTECT%20THE%20HARVEST-MO%20PAC/NAB_Protect_The_Harvest_PAC.pdf) on behalf of Protect the Harvest PAC. 

These forms are prescribed by the There are National Association of Broadcaster Forms. As such, it appears that KOMU's naming format for these files a couple of variations on the string `NAB`.

In [7]:
komu_agreements = komu_files.where(lambda row: is_agreement(row))

In [8]:
len(komu_agreements)

141

### Contracts

Next, there are contracts (aka, order forms) that specify the amount of paid by the advertiser for each ad, as well as the total for all the ads covered by the contract. [Here](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2016/State/PARSON%20FOR%20LT%20GOVERNOR%20OF%20MISSOURI/39109--1.pdf) is an example of a contract between KOMU and Governor Mike Parsons for $1,096.50 worth of ads that ran of July of 2016.

In [9]:
def is_komu_contract(row):
    return re.match(r'(.+)?\d{5}(--\d+)?', row['file_name'])

In [10]:
komu_contracts = komu_files.where(lambda row: is_contract(row))

In [11]:
len(komu_contracts)

599

### Others

There are handful of files left over.

In [12]:
len(komu_files) - (len(komu_agreements) + len(komu_contracts))

8

In [13]:
other_komu_files = komu_files.where(
    lambda row: not is_agreement(row) and not is_contract(row)
)

Here is what they look like.

In [14]:
other_komu_files \
    .select(['file_name']) \
    .print_table(max_rows=None, max_column_width=None)

| file_name                                           |
| --------------------------------------------------- |
| 38276                                               |
| Hillary Clinton Saturday Night Live July 23 2016    |
| Donald Trump Saturday Night Live November 7 2015    |
| MISSOURI FARMERS CARE 7-28-8-5                      |
| Hillary Clinton Saturday Night Live October 24 2015 |
| Hillary Clinton Saturday Night Live October 4 2015  |
| Bernie Sanders Saturday Night Live February 6 2016  |
| Bernie Sanders Saturday Night Live June 11 2016     |


In [15]:
other_komu_files \
    .select(['folder']) \
    .print_table(max_rows=None, max_column_width=None)

| folder                                                             |
| ------------------------------------------------------------------ |
| Political Files/2016/Federal/President/HILLARY FOR AMERICA         |
| Political Files/2016/Federal/President/HILLARY FOR AMERICA         |
| Political Files/2015/Federal/President/Donald Trump                |
| Political Files/2014/Non-Candidate Issue Ads/Missouri Farmers Care |
| Political Files/2015/Federal/President/Hillary Clinton             |
| Political Files/2015/Federal/President/Hillary Clinton             |
| Political Files/2016/Federal/President/Bernie Sanders              |
| Political Files/2016/Federal/President/Bernie Sanders              |


The "MISSOURI FARMERS CARE 7-28-8-5" is actually a [non-candidate issue ad contract](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2014/Non-Candidate%20Issue%20Ads/Missouri%20Farmers%20Care/MISSOURI_FARMERS_CARE_7-28-8-5.pdf). For some reason they threw in some extra dashes when they named this file.

The remaining files are all notices of free time provided by facilities to political candidates, such as the time [Donald Trump appeared on Saturday Night Live](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2015/Federal/President/Donald%20Trump/Donald_Trump_Saturday_Night_Live_November_7_2015.pdf) in Nov 2015. These notices are required by [Section 73.1943 (paragraph b)](https://www.law.cornell.edu/cfr/text/47/73.1943) of Federal Regulations. 

## KMIZ's Political Files

Now let's see if we can sort KMIZ's files into the same piles and/or identify new piles.

In [16]:
kmiz_files = fcc_files.where(lambda row: row['folder__entity'] == 'KMIZ')

In [17]:
len(kmiz_files)

1644

### Agreement Forms

In [18]:
kmiz_agreements = kmiz_files.where(lambda row: is_agreement(row))

In [19]:
len(kmiz_agreements)

123

### Contracts

In [20]:
kmiz_contracts = kmiz_files.where(lambda row: is_contract(row))

In [21]:
len(kmiz_contracts)

1519

### Others

In [22]:
len(kmiz_files) - (len(kmiz_agreements) + len(kmiz_contracts))

2

In [23]:
other_kmiz_files = kmiz_files.where(
    lambda row: not is_contract(row) and not is_agreement(row)
)

In [24]:
len(other_kmiz_files)

2

The first remaining file (located at [Political Files/2016/Terms and Disclosures/Disclosure_Statement.pdf](http://fcc-opif.s3.amazonaws.com/fcc_files/KMIZ/Political%20Files/2016/Terms%20and%20Disclosures/Disclosure_Statement.pdf))is a general statement about political advertising on KMIZ and affiliated channels, somewhat like a terms and conditions.

The second is [Political Files/2015/Non-Candidate Issue Ads/Missouri Propane/Non-Candidate Inquiry Form.pdf](http://fcc-opif.s3.amazonaws.com/fcc_files/KMIZ/Political%20Files/2015/Non-Candidate%20Issue%20Ads/Missouri%20Propane/Non-Candidate%20Inquiry%20Form/Missouri_Propane_Inquiry_Form.pdf) which sorta looks like and agreement form, but different.

## How well do these filters apply to all the other facilities?

In order to test this, we need the following counts for each facility:

- How many meet our criteria for an agreement (there should be many)
- How many meet our criteria for a contract (there should be even more)
- How many meet the criteria for being both (there should be none)
- How many don't meet the criteria for either (there should only be a handful)

In [38]:
by_facility = fcc_files.group_by('folder__entity').compute([
    ('is_agreement', agate.Formula(agate.Boolean(), lambda row: is_agreement(row))),
    ('is_contract', agate.Formula(agate.Boolean(), lambda row: is_contract(row))),
    ('is_both', agate.Formula(agate.Boolean(), lambda row: is_agreement(row) and is_contract(row))),
    ('is_neither', agate.Formula(agate.Boolean(), lambda row: not is_agreement(row) and not is_contract(row)))
]).aggregate([
    ('count_agreements', agate.Count(column_name='is_agreement', value=True)),
    ('count_contracts', agate.Count(column_name='is_contract', value=True)),
    ('count_both', agate.Count(column_name='is_both', value=True)),
    ('count_neither', agate.Count(column_name='is_neither', value=True)),
])

In [43]:
by_facility.order_by('count_neither', reverse=True).print_table(max_rows=None)

| folder__entity | count_agreements | count_contracts | count_both | count_neither |
| -------------- | ---------------- | --------------- | ---------- | ------------- |
| KRCG           |               78 |             264 |          0 |           814 |
| KRES           |                1 |               0 |          0 |           226 |
| KSSZ           |               28 |               3 |          0 |           202 |
| KWOS           |               28 |               6 |          0 |           186 |
| KCLR-FM        |               26 |               0 |          0 |           183 |
| KWIX           |                1 |               0 |          0 |           164 |
| KWRE           |                0 |               0 |          0 |           149 |
| KCMQ           |               19 |               0 |          0 |            91 |
| KFRU           |                0 |               0 |          0 |            73 |
| KATI           |               12 |               0 |          